1. 자신이 생각하는 가장 깔끔한 트랜스포머 모델을 파이토치로 구현하고 리얼한 데이터셋으로 학습과 추론을 해보세요. 

2. 1번의 코드를 바탕으로 트랜스포머의 얼개를 설명하는 보고서를 작성해보세요.

In [ ]:
import tiktoken
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

class MultiHeadAttention(nn.Module):
    # d_in= 임베딩 차원
    # d_out= 임베딩 차원
    # context_length= 한번에 처리 가능한 문맥크기
    # num_heads= 어텐션 헤드 개수
    # dropout= 드롭아웃 비율
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads

        # 출력차원을 어텐션헤드의 개수로 나눈다.        
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        # Linear도 결국 (d_in, d_out)크기의 행렬
        # 가중치 배열을 초기화하는 부분
        # (임베딩 크기, 임베딩 크기)만큼의 행렬을 생성한다.
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # 어텐션 출력을 병합하기 위한 레이어
        self.out_proj = nn.Linear(d_out, d_out) 

        self.dropout = nn.Dropout(dropout)

        # 마스킹을 위한 행렬
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    # 입력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # 가중치배열과 연산하여 Q, K, V를 만드는 부분
        # (6, 3) * (3, 3) = (6, 3)
        keys = self.W_key(x)  # Shape: (배치크기, 현재배치의 토큰 수, 토큰 임베딩 크기)
        queries = self.W_query(x)
        values = self.W_value(x)

        # 차원을 추가하여 head_dim 행렬을 나눈다.
        # 마지막 차원을 나눈다: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        # 1번, 2번 인덱스 차원의 행과 열을 바꾼다.
        # 각 헤드의 토큰을 병렬적으로 처리하기 위해 바꾸는 과정
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # queries : (b, num_heads, num_tokens, head_dim)
        # keys.transpose(2,3) : (b, num_heads, head_dim, num_tokens)
        # 출력 : (b, num_heads, num_tokens, num_tokens)
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # 마스킹 연산 수행
        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)


        # attn_weights : (b, num_heads, num_tokens, num_tokens)
        # values : (b, num_heads, num_tokens, head_dim)
        # 출력: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # contiguous() : view()함수를 사용하기 위해 벡터들을 메모리상에서 순서대로 배치시킴
        # view() : 분리된 헤드를 결합시켜 (b, num_tokens, d_out)와 같은 입력벡터와 동일한 차원으로 만든다.
        # 헤드 결합, self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        # 어텐션 블록이 여러 헤드를 통해 얻은 문맥 정보를 최종적으로 요약하고 압축하여 모델의 다음 단계로 전달하는 역할
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    # 정규화하여 입력과 같은 차원의 출력을 반환
    # scale, shift파라미터를 조정하여 성능향상
    def forward(self, x):
        # 소프트맥스 정규화 : 평균이 0, 분산이 1이 되도록 계산
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # 토큰 임베딩의 차원을 입력으로 받아서 4배로 차원을 확장시키고
        # GELU함수 적용
        # 4배 확장된 입력을 받아서 원래 차원으로 다시 복귀시킴
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    # 입력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
    def forward(self, x):
        ##### 멀티헤드어텐션 연산 부분
        # 잔차연결값 저장
        shortcut = x
        # 정규화
        x = self.norm1(x)
        # 멀티헤드어텐션 연산 수행
        x = self.att(x)   # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        # 드롭아웃
        x = self.drop_shortcut(x)
        # 잔차연결
        x = x + shortcut  # Add the original input back

        ##### 피드포워드 연산 부분
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x) # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # 각 단어들을 임베딩 토큰으로 변경하기 위한 조회 테이블
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Sequential: 배열에 담긴 레이어를 묶어서 한번에 관리하기 위함
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape

        # 모델입력 : (배치크기 :한번에 병렬적으로 추론할 문장의 수, 현재배치의 토큰 수)
        # 예를들어 (2:배치크기, 4: 토큰 수)이 입력인 경우 (2, 4, 임베딩 크기)로 벡터를 변환시킴
        # 토큰 임베딩 이후 포지셔널 임베딩 벡터와 합산
        # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)

        # 멀티헤드어텐션 연산 수행
        # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        x = self.trf_blocks(x)

        # 마지막 정규화층 연산
        # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        x = self.final_norm(x)

        # out_head = Linear(단어임베딩 크기, 단어사전 크기)
        # 각각의 단어를 선형층을 통해 단어 마다의 확률로 변환시키는 과정
        # out_head의 선형층을 거치면서 마지막 차원의 크기가 (단어임베딩 크기 -> 단어사전 크기) 로 확장된다.
        # 모델출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 각 토큰마다 단어사전의 확률분포)
        logits = self.out_head(x)
        return logits


# idx : 초기 입력
# max_new_tokens : 생성할 최대 토큰
# context_size : 모델이 한 번에 볼 수 있는 문맥크기
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # max_new_tokens개 만큼의 토큰이 생성 될 때까지 반복
    for _ in range(max_new_tokens):

        # 모델이 한번에 처리 가능한 문맥 크기를 넘는 경우, 앞부분을 자르기 위한 코드
        idx_cond = idx[:, -context_size:]

        with torch.no_grad():
            # 모델입력 : (배치크기 :한번에 병렬적으로 추론할 문장의 수, 현재배치의 토큰 수)
            # 모델출력 : (배치크기 :한번에 병렬적으로 추론할 문장의 수), 현재배치의 토큰 수, 각 토큰마다 단어사전의 확률분포)
            logits = model(idx_cond)

        # 마지막 단어의 확률분포만 남김
        logits = logits[:, -1, :]

        # 가장 높은 확률을 가진 단어만 남김
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # 가장 높은 확률을 가진 단어를 기존 문맥에 이어붙임
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


def main():
    GPT_CONFIG_124M = {
        "vocab_size": 50257,     # Vocabulary size
        "context_length": 1024,  # Context length
        "emb_dim": 768,          # Embedding dimension
        "n_heads": 12,           # Number of attention heads
        "n_layers": 12,          # Number of layers
        "drop_rate": 0.1,        # Dropout rate
        "qkv_bias": False        # Query-Key-Value bias
    }

    torch.manual_seed(123)
    model = GPTModel(GPT_CONFIG_124M)
    model.eval()  # disable dropout

    start_context = "Hello, I am"

    tokenizer = tiktoken.get_encoding("gpt2")
    encoded = tokenizer.encode(start_context)

    # 차원의 0번 인덱스의 새로운 차원을 추가 ( ,4) -> (1, 4)
    # 모델은 배치로 실행되기 때문에 배치 차원을 추가하는 과정이다.
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)

    print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
    print("\nInput text:", start_context)
    print("Encoded input text:", encoded)
    print("encoded_tensor.shape:", encoded_tensor.shape)

    out = generate_text_simple(
        model=model,
        idx=encoded_tensor,
        max_new_tokens=10,
        context_size=GPT_CONFIG_124M["context_length"]
    )


    # 2차원으로 나온 결과의 0번 인덱스 차원을 제거하여 1차원으로 변환
    decoded_text = tokenizer.decode(out.squeeze(0).tolist())

    print(f"\n\n{50*'='}\n{22*' '}OUT\n{50*'='}")
    print("\nOutput:", out)
    print("Output length:", len(out[0]))
    print("Output text:", decoded_text)


if __name__ == "__main__":
    main()


                      IN

Input text: Hello, I am
Encoded input text: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


                      OUT

Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267,
         49706, 43231, 47062, 34657]])
Output length: 14
Output text: Hello, I am Featureiman Byeswickattribute argue logger Normandy Compton analogous


In [ ]:
length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

class MultiHeadAttention(nn.Module):
    # d_in= 임베딩 차원
    # d_out= 임베딩 차원
    # context_length= 한번에 처리 가능한 문맥크기
    # num_heads= 어텐션 헤드 개수
    # dropout= 드롭아웃 비율
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads

        # 출력차원을 어텐션헤드의 개수로 나눈다.        
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        # Linear도 결국 (d_in, d_out)크기의 행렬
        # 가중치 배열을 초기화하는 부분
        # (임베딩 크기, 임베딩 크기)만큼의 행렬을 생성한다.
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # 어텐션 출력을 병합하기 위한 레이어
        self.out_proj = nn.Linear(d_out, d_out) 

        self.dropout = nn.Dropout(dropout)

        # 마스킹을 위한 행렬
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    # 입력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # 가중치배열과 연산하여 Q, K, V를 만드는 부분
        # (6, 3) * (3, 3) = (6, 3)
        keys = self.W_key(x)  # Shape: (배치크기, 현재배치의 토큰 수, 토큰 임베딩 크기)
        queries = self.W_query(x)
        values = self.W_value(x)

        # 차원을 추가하여 head_dim 행렬을 나눈다.
        # 마지막 차원을 나눈다: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        # 1번, 2번 인덱스 차원의 행과 열을 바꾼다.
        # 각 헤드의 토큰을 병렬적으로 처리하기 위해 바꾸는 과정
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # queries : (b, num_heads, num_tokens, head_dim)
        # keys.transpose(2,3) : (b, num_heads, head_dim, num_tokens)
        # 출력 : (b, num_heads, num_tokens, num_tokens)
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # 마스킹 연산 수행
        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)


        # attn_weights : (b, num_heads, num_tokens, num_tokens)
        # values : (b, num_heads, num_tokens, head_dim)
        # 출력: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # contiguous() : view()함수를 사용하기 위해 벡터들을 메모리상에서 순서대로 배치시킴
        # view() : 분리된 헤드를 결합시켜 (b, num_tokens, d_out)와 같은 입력벡터와 동일한 차원으로 만든다.
        # 헤드 결합, self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        # 어텐션 블록이 여러 헤드를 통해 얻은 문맥 정보를 최종적으로 요약하고 압축하여 모델의 다음 단계로 전달하는 역할
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    # 정규화하여 입력과 같은 차원의 출력을 반환
    # scale, shift파라미터를 조정하여 성능향상
    def forward(self, x):
        # 소프트맥스 정규화 : 평균이 0, 분산이 1이 되도록 계산
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # 토큰 임베딩의 차원을 입력으로 받아서 4배로 차원을 확장시키고
        # GELU함수 적용
        # 4배 확장된 입력을 받아서 원래 차원으로 다시 복귀시킴
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)


class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    # 입력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
    def forward(self, x):
        ##### 멀티헤드어텐션 연산 부분
        # 잔차연결값 저장
        shortcut = x
        # 정규화
        x = self.norm1(x)
        # 멀티헤드어텐션 연산 수행
        x = self.att(x)   # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        # 드롭아웃
        x = self.drop_shortcut(x)
        # 잔차연결
        x = x + shortcut  # Add the original input back

        ##### 피드포워드 연산 부분
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x) # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # 각 단어들을 임베딩 토큰으로 변경하기 위한 조회 테이블
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # Sequential: 배열에 담긴 레이어를 묶어서 한번에 관리하기 위함
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape

        # 모델입력 : (배치크기 :한번에 병렬적으로 추론할 문장의 수, 현재배치의 토큰 수)
        # 예를들어 (2:배치크기, 4: 토큰 수)이 입력인 경우 (2, 4, 임베딩 크기)로 벡터를 변환시킴
        # 토큰 임베딩 이후 포지셔널 임베딩 벡터와 합산
        # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)

        # 멀티헤드어텐션 연산 수행
        # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        x = self.trf_blocks(x)

        # 마지막 정규화층 연산
        # 출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 토큰 임베딩 크기)
        x = self.final_norm(x)

        # out_head = Linear(단어임베딩 크기, 단어사전 크기)
        # 각각의 단어를 선형층을 통해 단어 마다의 확률로 변환시키는 과정
        # out_head의 선형층을 거치면서 마지막 차원의 크기가 (단어임베딩 크기 -> 단어사전 크기) 로 확장된다.
        # 모델출력 : (배치크기 :한번에 병렬적으로 추론할 단어의 수, 현재배치의 토큰 수, 각 토큰마다 단어사전의 확률분포)
        logits = self.out_head(x)
        return logits


# idx : 초기 입력
# max_new_tokens : 생성할 최대 토큰
# context_size : 모델이 한 번에 볼 수 있는 문맥크기
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # max_new_tokens개 만큼의 토큰이 생성 될 때까지 반복
    for _ in range(max_new_tokens):

        # 모델이 한번에 처리 가능한 문맥 크기를 넘는 경우, 앞부분을 자르기 위한 코드
        idx_cond = idx[:, -context_size:]

        with torch.no_grad():
            # 모델입력 : (배치크기 :한번에 병렬적으로 추론할 문장의 수, 현재배치의 토큰 수)
            # 모델출력 : (배치크기 :한번에 병렬적으로 추론할 문장의 수), 현재배치의 토큰 수, 각 토큰마다 단어사전의 확률분포)
            logits = model(idx_cond)

        # 마지막 단어의 확률분포만 남김
        logits = logits[:, -1, :]

        # 가장 높은 확률을 가진 단어만 남김
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # 가장 높은 확률을 가진 단어를 기존 문맥에 이어붙임
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


def main():
    GPT_CONFIG_124M = {
        "vocab_size": 50257,     # Vocabulary size
        "context_length": 1024,  # Context length
        "emb_dim": 768,          # Embedding dimension
        "n_heads": 12,           # Number of attention heads
        "n_layers": 12,          # Number of layers
        "drop_rate": 0.1,        # Dropout rate
        "qkv_bias": False        # Query-Key-Value bias
    }

    torch.manual_seed(123)
    model = GPTModel(GPT_CONFIG_124M)
    model.eval()  # disable dropout

    start_context = "Hello, I am"

    tokenizer = tiktoken.get_encoding("gpt2")
    encoded = tokenizer.encode(start_context)

    # 차원의 0번 인덱스의 새로운 차원을 추가 ( ,4) -> (1, 4)
    # 모델은 배치로 실행되기 때문에 배치 차원을 추가하는 과정이다.
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)

    print(f"\n{50*'='}\n{22*' '}IN\n{50*'='}")
    print("\nInput text:", start_context)
    print("Encoded input text:", encoded)
    print("encoded_tensor.shape:", encoded_tensor.shape)

    out = generate_text_simple(
        model=model,
        idx=encoded_tensor,
        max_new_tokens=10,
        context_size=GPT_CONFIG_124M["context_length"]
    )


    # 2차원으로 나온 결과의 0번 인덱스 차원을 제거하여 1차원으로 변환
    decoded_text = tokenizer.decode(out.squeeze(0).tolist())

    print(f"\n\n{50*'='}\n{22*' '}OUT\n{50*'='}")
    print("\nOutput:", out)
    print("Output length:", len(out[0]))
    print("Output text:", decoded_text)


if __name__ == "__main__":
    main()

In [4]:
import matplotlib.pyplot as plt
import os
import requests
import torch
import tiktoken


# Import from local files
from gpt_module import GPTModel, create_dataloader_v1, generate_text_simple


def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text)
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # add batch dimension
    return encoded_tensor


def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)  # remove batch dimension
    return tokenizer.decode(flat.tolist())


def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
        decoded_text = token_ids_to_text(token_ids, tokenizer)
        print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()


def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen = 0
    global_step = -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()  # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward()  # Calculate loss gradients
            optimizer.step()  # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots()

    # Plot training and validation loss against epochs
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(epochs_seen, val_losses, linestyle="-.", label="Validation loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")

    # Create a second x-axis for tokens seen
    ax2 = ax1.twiny()  # Create a second x-axis that shares the same y-axis
    ax2.plot(tokens_seen, train_losses, alpha=0)  # Invisible plot for aligning ticks
    ax2.set_xlabel("Tokens seen")

    fig.tight_layout()  # Adjust layout to make room
    # plt.show()


def main(gpt_config, settings):

    torch.manual_seed(123)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ##############################
    # Download data if necessary
    ##############################

    file_path = "the-verdict.txt"
    url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

    if not os.path.exists(file_path):
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        text_data = response.text
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
    else:
        with open(file_path, "r", encoding="utf-8") as file:
            text_data = file.read()
    ##############################
    # Initialize model
    ##############################

    model = GPTModel(gpt_config)
    model.to(device)  # no assignment model = model.to(device) necessary for nn.Module classes
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=settings["learning_rate"], weight_decay=settings["weight_decay"]
    )

    ##############################
    # Set up dataloaders
    ##############################

    # Train/validation ratio
    train_ratio = 0.90
    split_idx = int(train_ratio * len(text_data))

    train_loader = create_dataloader_v1(
        text_data[:split_idx],
        batch_size=settings["batch_size"],
        max_length=gpt_config["context_length"],
        stride=gpt_config["context_length"],
        drop_last=True,
        shuffle=True,
        num_workers=0
    )

    val_loader = create_dataloader_v1(
        text_data[split_idx:],
        batch_size=settings["batch_size"],
        max_length=gpt_config["context_length"],
        stride=gpt_config["context_length"],
        drop_last=False,
        shuffle=False,
        num_workers=0
    )

    ##############################
    # Train model
    ##############################

    tokenizer = tiktoken.get_encoding("gpt2")

    train_losses, val_losses, tokens_seen = train_model_simple(
        model, train_loader, val_loader, optimizer, device,
        num_epochs=settings["num_epochs"], eval_freq=5, eval_iter=1,
        start_context="Every effort moves you", tokenizer=tokenizer
    )

    return train_losses, val_losses, tokens_seen, model


if __name__ == "__main__":

    GPT_CONFIG_124M = {
        "vocab_size": 50257,    # Vocabulary size
        "context_length": 256,  # Shortened context length (orig: 1024)
        "emb_dim": 768,         # Embedding dimension
        "n_heads": 12,          # Number of attention heads
        "n_layers": 12,         # Number of layers
        "drop_rate": 0.1,       # Dropout rate
        "qkv_bias": False       # Query-key-value bias
    }

    OTHER_SETTINGS = {
        "learning_rate": 5e-4,
        "num_epochs": 10,
        "batch_size": 2,
        "weight_decay": 0.1
    }

    ###########################
    # Initiate training
    ###########################

    train_losses, val_losses, tokens_seen, model = main(GPT_CONFIG_124M, OTHER_SETTINGS)

    ###########################
    # After training
    ###########################

    # Plot results
    epochs_tensor = torch.linspace(0, OTHER_SETTINGS["num_epochs"], len(train_losses))
    plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)
    plt.savefig("loss.pdf")

    # Save and load model
    torch.save(model.state_dict(), "model.pth")
    model = GPTModel(GPT_CONFIG_124M)
    model.load_state_dict(torch.load("model.pth", weights_only=True))


ModuleNotFoundError: No module named 'gpt_module'

3. 주어진 데이터셋 중 원하는 데이터셋을 사용해 원하는 사전 훈련 모델(CNN)을 훈련시키고, 연구보고서를 작성해 보세요.
- (필수) Test Accuracy, Test Loss 출력 값과 성능 시각화(그래프)를 포함한 연구 및 분석 과정에 대한 내용이 있어야합니다.
- (선택) 위 성공 조건을 달성했다면. 실제 데이터를 입력해 예측 또는 분류 결과 출력 후 분석한 내용도 작성해보세요.


아래 항목을 포함하여 모델의 학습 및 성능을 분석하는 내용을 중심으로 작성하세요.
- **Train Accuracy / Train Loss / Test Accuracy / Test Loss**
- **성능 시각화 (정확도·손실 곡선, 혼동 행렬 등)**
- **결과 해석 및 개선 방향 제시**
- **모델 설계 및 파라미터 선택 이유**

단순히 수치를 나열하지 말고, “왜 이런 결과가 나왔는가”, “무엇을 시도했고, 어떤 교훈을 얻었는가”**를 논리적으로 설명해 주세요.

In [ ]:
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models
import torch.nn as nn
import torch
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.data import Subset

# A100 GPU 최적화 설정
torch.backends.cudnn.benchmark = True  # 최적의 알고리즘 자동 선택
torch.backends.cuda.matmul.allow_tf32 = True  # TF32 활성화 (A100 특화)
torch.backends.cudnn.allow_tf32 = True



def get_CIFAR10_data_loader(ratio:int = 1.0)-> tuple[DataLoader, DataLoader]:
    """
    CIFAR10데이터셋을 가져오는 함수 (전체 훈련셋 : 50000개, 전체 테스트셋 : 10000개)

    Args:
        ratio: 전체 데이터셋 대비 비중

    Returns:
        (train_loader, test_loader)
    """
    TRAINSET_COUNT = 50000

    # 1. 전처리 정의 (GPU 친화적)
    transform_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),  # 데이터 증강
        transforms.RandomCrop(224, padding=4),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])

    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])

    # 데이터셋 로드
    trainset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train
    )

    # 데이터 서브셋 생성
    indices = list(range(int(TRAINSET_COUNT * ratio)))
    subset_trainset = Subset(trainset, indices)

    testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test
    )

    # 3. DataLoader 생성 (A100 최적화)
    # A100은 대용량 배치를 효율적으로 처리
    trainloader = torch.utils.data.DataLoader(
        subset_trainset,
        batch_size=64,  # A100에 최적화된 큰 배치 사이즈
        shuffle=True,
        num_workers=12,  # A100 시스템의 CPU 코어 활용
        pin_memory=True,  # GPU 전송 속도 향상
        prefetch_factor=4,  # 미리 가져올 배치 수
        persistent_workers=True  # Worker 재사용
    )

    testloader = torch.utils.data.DataLoader(
        testset,
        batch_size=512,  # 평가 시 더 큰 배치
        shuffle=False,
        num_workers=8,
        pin_memory=True,
        prefetch_factor=2,
        persistent_workers=True
    )

    return trainloader, testloader

def get_custom_resnet50(class_count:int = 10) -> models.ResNet:
    """
    resnet의 분류클래스를 변경하여 리턴하는 함수

    Args:
        클래스의 개수

    Returns:
        ResNet모델
    """
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    # 모델의 마지막 완전연결층 레이어 변경
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, class_count)

    return model

def get_custom_vgg16(class_count:int = 10) -> models.vgg16:
    """
    VGG16의 분류클래스를 변경하여 리턴하는 함수

    Args:
        클래스의 개수

    Returns:
        VGG16모델
    """
    model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    in_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(in_features, 10)

    return model


def test_model(model, loss_function, test_loader, device):
    """
    모델을 테스트 하는 함수

    Args:
        model : 테스트 모델,
        loss_function : 손실함수
        test_loader : 테스트 데이터셋,
        device : 테스트를 수행할 디바이스

    Returns:
        학습 오차율, 정확도(%)
    """
    model.to(device)
    model.eval() # 👈 모델을 평가 모드로 설정
    total = 0
    correct = 0
    with torch.no_grad(): # 👈 기울기 계산 비활성화
        running_loss = 0.0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            loss = loss_function(outputs, labels)
            running_loss += loss.item()
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    return (running_loss, accuracy)


def train_model(model, optimizer, loss_function, train_loader, test_loader, device):
  """
  모델을 훈련 하는 함수

  Args:
      model : 테스트 모델,
      optimizer : 옵티마이저,
      loss_function : 손실함수,
      train_loader : 훈련 데이터셋
      test_loader : 테스트 데이터셋,
      device : 훈련을 수행할 디바이스

  Returns:
      list : 각 에포크마다 학습 오차율, 정확도(%)
  """

  model.to(device)

  print("Starting Training...")
  num_epochs = 5  # 👈 전체 데이터셋을 5번 반복 학습 (시간이 오래 걸릴 수 있습니다)
  result = []
  for epoch in range(num_epochs):
      # Dropout, BatchNorm 등이 학습 모드로 동작합니다.
      model.train()

      # train_loader에서 배치(batch) 단위로 데이터를 가져옵니다.
      for i, (images, labels) in enumerate(train_loader):
          # 1. 데이터를 디바이스로 이동
          images = images.to(device, non_blocking=True)
          labels = labels.to(device, non_blocking=True)

          # 2. (필수) 옵티마이저의 기울기(gradient)를 0으로 초기화
          optimizer.zero_grad(set_to_none=True)

          # 3. 순전파(Forward pass): 모델에 이미지를 입력하여 출력(로짓) 계산
          outputs = model(images)

          # 4. 손실(Loss) 계산
          loss = loss_function(outputs, labels)

          # 5. (필수) 역전파(Backward pass): 손실에 대한 기울기 계산
          loss.backward()

          # 6. (필수) 옵티마이저 실행: 계산된 기울기를 바탕으로 가중치 업데이트
          optimizer.step()

      # --- 한 Epoch이 끝날 때마다 테스트 정확도 평가 ---
      epoch_loss, epoch_accuracy = test_model(model, loss_function, test_loader, device)
      result.append((epoch_loss, epoch_accuracy))
      print(f"에포크 {epoch+1} 학습 완료, 정확도 : {epoch_accuracy}, 손실 : {epoch_loss}")

  return result


def make_loss_accuracy_graph(losses:list[tuple[str, list[float]]], accrucies:list[tuple[str, float]]):
    """
    손실, 정확도 그래프를 그려주는 함수

    Args:
        losses : (훈련조건, 훈련결과) 를 담은 리스트
        accrucies : (훈련조건, 훈련결과) 를 담은 리스트

    Returns:
        없음
    """

    fig, axes = plt.subplots(nrows=1, ncols=2)

    # 손실그래프 생성
    for loss in losses:
        label, values = loss
        axes[0].plot([i for i in range (1, len(values)+1)], values, label=label)

    axes[0].set_title("loss graph")
    axes[0].set_xlabel("epoch")
    axes[0].set_ylabel("loss")
    axes[0].grid(True, linestyle='--', alpha=0.6)
    axes[0].legend()


    # 정확도 그래프 생성
    for accuracy in accrucies:
        label, values = accuracy
        axes[1].plot([i for i in range (1, len(values)+1)], values, label=label)

    axes[1].set_title("accuracy graph")
    axes[1].set_xlabel("epoch")
    axes[1].set_ylabel("accuracy(%)")
    axes[1].grid(True, linestyle='--', alpha=0.6)
    axes[1].legend()

    fig.tight_layout() # 서브플롯들이 겹치지 않도록 간격조절
    plt.show()

# 이미지 출력하기
def print_img(data, label):
    img_grid = data * 0.5 + 0.5
    np_grid = img_grid.numpy()
    np_grid_rgb = np.transpose(np_grid,(1,2,0)) # 기존 (채널, 높이, 너비) -> (높이, 너비, 채널)

    print(label)

    plt.figure(figsize=(10, 2)) # 가로 10, 세로 2 인치
    plt.imshow(np_grid_rgb)
    plt.axis('off') # 축 정보 끄기
    plt.show()




In [ ]:
import numpy as np
import torch
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

train_loader, test_loader = get_CIFAR10_data_loader(ratio=0.1)
model = get_custom_resnet50(class_count=10)

loss_function = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(
#     model.parameters(),
#     lr=1e-3,
#     weight_decay=1e-4  # 정규화
# )
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
no_train = [test_model(model, loss_function, test_loader, device)] * 5
train = train_model(model, optimizer, loss_function, train_loader, test_loader, device)

# 그래프 생성
losses = [("no_train", list(map(lambda t:t[0], no_train))), ("train", list(map(lambda t:t[0], train)))]
accrucies = [("no_train", list(map(lambda t:t[1], no_train))), ("train", list(map(lambda t:t[1], train)))]
make_loss_accuracy_graph(losses=losses ,accrucies=accrucies)